In [6]:
import numpy as np

## コーパスの前処理

In [5]:
text = 'You say goobye and I say hello.'

text = text.lower()
text = text.replace('.', ' .')
print(text)

words = text.split(sep=' ')
print('split後:', words)

you say goobye and i say hello .
split後: ['you', 'say', 'goobye', 'and', 'i', 'say', 'hello', '.']


In [22]:
# 単語のid化

# 対応するdictを作成
word_to_id = {}
id_to_word = {}

for word in words:
    if word not in word_to_id:
        new_id = len(word_to_id)
        word_to_id[word] = new_id
        id_to_word[new_id] = word

print(word_to_id)
print(id_to_word)

{'you': 0, 'say': 1, 'goobye': 2, 'and': 3, 'i': 4, 'hello': 5, '.': 6}
{0: 'you', 1: 'say', 2: 'goobye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


In [29]:
# 数値として扱いやすいようにwordsを単語idに変換
corpus = [word_to_id[word] for word in words]

# np.array
corpus = np.array(corpus)
corpus

array([0, 1, 2, 3, 4, 1, 5, 6])

In [31]:
# 上記の処理を関数化

def preprocess(texts):
    texts = texts.lower()
    texts = texts.replace('.', ' .')
    words = texts.split(sep=' ')

    word_to_id = {}
    id_to_word = {}

    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word

    corpus = np.array([word_to_id[w] for w in words])
    
    return corpus, word_to_id, id_to_word

In [32]:
# 関数チェック
text = 'You say goobye and I say hello.'

corpus, word_to_id, id_to_word = preprocess(text)

・コーパスを扱う準備はこれで終わり。  
・次からはこのコーパスを使用して、単語の意味を抽出をする。　-> 「カウントベースの手法」  
・単語の意味を的確に捉えたベクトル表現を目指す。 (ex: 色のRGBのようなベクトル表現  ->  分散表現  
・単語の分散表現：単語を固定長のベクトルで表現、密なベクトル。  
<br>
・単語の意味は周囲の単語によって形成される

## 共起行列

In [39]:
# テキストと前処理
text = 'You say goobye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
print(corpus)
print(word_to_id)

[0 1 2 3 4 1 5 6]
{'you': 0, 'say': 1, 'goobye': 2, 'and': 3, 'i': 4, 'hello': 5, '.': 6}


In [40]:
# 共起行列 (window size = 1)


C = np.array([
    [0, 1, 0, 0, 0, 0, 0],
    [1, 0, 1, 0, 1, 1, 0],
    [0, 1, 0, 1, 0, 0, 0],
    [0, 0, 1, 0, 1, 0, 0],
    [0, 1, 0, 1, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 1, 0]
], dtype=np.int32)

In [71]:
# コーパスから共起行列を作る関数

def create_co_matrix(corpus, vocab_size, window_size=1):
    corpus_size = len(corpus)
    co_matrix = np.zeros([corpus_size, corpus_size], dtype=np.int32)

    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size + 1):
            left_idx = idx - i
            right_idx = idx + i

            if left_idx >= 0:
                left_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] = 1
            
            
            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] = 1

    return co_matrix

In [72]:
# コサイン類似度
def cos_similarity(x, y, eps=1e-8):
    nx = x/(np.sqrt(np.sum(x**2))+eps)
    ny = y/(np.sqrt(np.sum(y**2))+eps)
    return np.dot(nx, ny)    

In [81]:
# コサイン類似度のチェック
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)

# youとIの類似度
c0 = C[word_to_id['you']]
c1 = C[word_to_id['i']]

print(c0)
print(c1)
print(cos_similarity(c0, c1))





[0 1 0 0 0 0 0 0]
[0 1 0 1 0 0 0 0]
0.7071067691154799


# 